In [17]:
import os
import glob
import numpy as np
import pandas as pd

def get_prefix_from_path(file_path):
    if not file_path:
        return 'default'

    base_name = os.path.splitext(os.path.basename(file_path))[0]
    parts = base_name.split('_')
    
    if len(parts) >= 2:
        return '_'.join(parts[:2])
    else:
        return base_name

def activity(input_file_path1,input_file_path2,input_file_path3,input_file_path4,input_file_path5):

    df=pd.read_csv('activity.csv')
    df=pd.DataFrame(df)
    #采用contributor busfactor inact new
    #bus不可失去即核心
    # 防止没有搜寻到路径，则将该方面评分设置为0
    data_con = pd.read_csv(input_file_path1)['指标值'] if input_file_path1 else np.zeros(300, dtype=int)
    data_bus = pd.read_csv(input_file_path2)['指标值'] if input_file_path2 else np.zeros(300, dtype=int)
    data_new = pd.read_csv(input_file_path3)['指标值'] if input_file_path3 else np.zeros(300, dtype=int)
    data_inact = pd.read_csv(input_file_path4)['指标值'] if input_file_path4 else np.zeros(300, dtype=int)
    data_part = pd.read_csv(input_file_path5)['指标值'] if input_file_path5 else np.zeros(300, dtype=int)

    s_con=[]
    s_bus=[]
    s_new=[]
    s_inact=[]

    for i in range(len(data_con)): 
        if data_bus[i]!=0:
            s_con.append(data_con[i]/data_bus[i])        # 贡献者 占总人数的比例
        if data_part[i]!=0:
            s_bus.append(data_bus[i]/data_part[i])       # bus_factor 占总参与人数的比例
            s_new.append(data_new[i]/data_part[i])       # 新贡献者 占总人数的比例
            s_inact.append(data_inact[i]/data_part[i])   # 不活跃的贡献者 占总人数的比例

    avg1=np.mean(s_bus)
    avg2=np.mean(s_inact)
    avg3=np.mean(s_new)
    avg4=np.mean(s_con)
    avg5=1-avg3
    avg6=1-avg1
    final=avg6/4+avg2/4+avg5/4+avg4/4
    # 获取仓库名
    prefix=get_prefix_from_path(input_file_path1)
    row_exists=(df['指标名称']==prefix).any()

    if row_exists:
        df.loc[df['指标名称'] == prefix, '指标值']=final*10

    # 保存更改回 CSV 文件
    df.to_csv('activity.csv', index=False)



def get_prefix_and_rest(filename):

    base_name = os.path.basename(filename)
    # 分离文件名和扩展名
    name_part, ext_part = os.path.splitext(base_name) 
    # 按下划线分割文件名部分，并确保至少有两个'_'
    parts = name_part.split('_')

    if len(parts) > 1:
        # 获取前两个'_'之间的部分作为前缀
        prefix = '_'.join(parts[:2])
        # 剩余部分加上原始扩展名作为后缀
        rest = '_'.join(parts[2:]) + ext_part if len(parts) > 2 else ext_part
    else:
        # 如果没有足够的'_'，则返回整个文件名作为前缀，空字符串作为后缀
        prefix = name_part
        rest = ext_part
    
    return prefix, rest

def process_files_with_same_prefix(base_dir):
    # 定义需要查找的文件后缀
    file_suffixes = [
        'contributor.csv', 'bus_factor.csv', 'new_contributors.csv',
        'inactive_contributors.csv', 'participants.csv',
        'change_request_resolution_duration.csv',
        'change_request_response_time.csv', 'change_requests.csv',
        'change_requests_accepted.csv'
    ]
    
    # 获取所有csv文件
    all_files = glob.glob(os.path.join(base_dir, '*.csv'))
    # 创建一个字典来存储每个前缀对应的文件路径列表
    files_dict = {}
    
    for file in all_files:
        
        # 获取仓库和后缀的部分作为前缀
        prefix, suffix = get_prefix_and_rest(os.path.basename(file))
        
        key = next((s for s in file_suffixes if s==suffix), None)
        if key:  # prefix是仓库名，key是文件后缀匹配，file是路径
            if prefix not in files_dict:
                files_dict[prefix] = {}
            files_dict[prefix][key] = file
    # 遍历字典并调用activity函数
    for prefix, paths in files_dict.items():
        # 调用 activity 函数，传递对应前缀的所有文件路径
        input_file_path1=paths.get('contributor.csv')
        input_file_path2=paths.get('bus_factor.csv')
        input_file_path3=paths.get('new_contributors.csv')
        input_file_path4=paths.get('inactive_contributors.csv')
        input_file_path5=paths.get('participants.csv')
        input_file_path6=paths.get('change_request_resolution_duration.csv')
        input_file_path7=paths.get('change_request_response_time.csv')
        input_file_path8=paths.get('change_requests.csv')            
        input_file_path9=paths.get('change_requests_accepted.csv')

        activity(input_file_path1,input_file_path2,input_file_path3,input_file_path4,input_file_path5)
# 假设 base_dir 是你的文件夹路径
base_dir='./最终数据处理'
process_files_with_same_prefix(base_dir)